In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from Deepfunction import *

In [ ]:
class Model:
    """Model Declaration"""
    def __init__(self):
        """Constructor"""
        self.LayerList = []
        
    def add(self, layer):
        """Layer adding function"""
        self.LayerList.append(layer)
    
    def fit(self, X_train, y_train, X_test, y_test, epochs, alpha):
        """Fitting function"""
        num = X_train.shape[0]
        
        for i in range(epochs):
            for x, y in zip(X_train, y_train):
                for layer in self.LayerList:
                    x = layer.forward(x)

                for layer in self.LayerList[::-1]:
                    y = layer.backprop(y)

                for layer in self.LayerList:
                    layer.update(alpha, num)
                    
            print(f"{i+1} : {self.score(X_test, y_test)}")
            
    def score(self, X_test, y_test):
        """Score function"""
        score = []

        for x, y in zip(X_test, y_test):
            for layer in self.LayerList:
                x = layer.forward(x)

            score.append(np.argmax(x) == np.argmax(y))

        return np.mean(score)
    
class Dense:
    """Fully Connected layer"""
    def __init__ (self, pre_neurons, neurons, activation = 'sigmoid', optimization = None,
                  num = 0, layer = 'hidden'):
        """Constructor"""
        self.W = np.random.randn(neurons, pre_neurons) * np.sqrt(1.0 / neurons)
        self.b = np.random.randn(neurons) * np.sqrt(1.0 / neurons)
        self.activation = activation
        self.num = num
        self.layer = layer
        
    def forward(self, a_in):
        """Forward propagation"""
        self.a_in = a_in
        self.z = self.W @ self.a_in + self.b
        
        if self.activation == 'sigmoid':
            self.a_out = sigmoid(self.z)
        
        elif self.activation == 'relu':
            self.a_out = relu(self.z)
            
        elif self.activation == 'softmax':
            self.a_out = softmax(self.z)
        
        return self.a_out
    
    def backprop(self, grad_a):
        """Back propagation"""
        if self.layer == 'output': # layer feature
            grad_a = (self.a_out - grad_a) / len(self.a_out)

        if self.activation == 'sigmoid': # activation feature
            self.grad_b = grad_a * d_sigmoid(self.z)
            
        elif self.activation == 'relu':
            self.grad_b = grad_a * d_relu(self.z)
        
        elif self.activation == 'softmax':
            self.grad_b = grad_a
        
        self.grad_W = np.outer(self.grad_b, self.a_in)
        self.grad_a = self.W.T @ self.grad_b

        return self.grad_a
    
    def update(self, alpha, data_num):
        """Parameters update"""
        self.W = self.W - alpha * self.grad_W / data_num
        self.b = self.b - alpha * self.grad_b / data_num

    def __str__(self):
        """Print weight and bias"""
        return (f"{self.num}'s Layer W\n{self.W}\n\n{self.num}'s Layer b\n{self.b}")


In [ ]:
np.random.seed(45)
dataset = pd.read_csv('./MNIST_preprocessed.csv', sep=',', header=None).values

X = dataset[:, 0:784]
Y = dataset[:, 784:]
X_train, X_test = X[0:250,], X[250:,]
y_train, y_test = Y[0:250,], Y[250:,]

"""
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_class = 10
X_train = X_train[:10000].reshape(-1,28*28).astype(np.float32)
X_test = X_test[:3000].reshape(-1,28*28).astype(np.float32)

y_train = y_train[:10000]
y_test = y_test[:3000]
y_train = OneHot(y_train, num_class)

plt.imshow(X_train[0].reshape(28,28),cmap='gray')
"""

model = Model()
model.add(Dense(784,128,num=1))
model.add(Dense(128,64,num=2))
Densout = Dense(64,10,num=3,layer='output')
model.add(Densout)
model.fit(X_train, y_train, X_test, y_test, 10, 300)
print(model.score(X_test, y_test))